In [ ]:
import cv2
import numpy as np

In [ ]:
def read_image(IMG_DIR, IMG_NAME_1, IMG_NAME_2):
    img_1 = cv2.imread(IMG_DIR + IMG_NAME_1)
    img_2 = cv2.imread(IMG_DIR + IMG_NAME_2)
    cv2.imshow('image1',img_1)
    cv2.imshow('image2',img_2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img_1, img_2

In [ ]:
IMG_DIR = 'image_0/'


In [ ]:
T = np.zeros((3, 1))
R = np.eye(3)
for root,dirs,files in os.walk(DATA_DIR):
    for i, f in enumerate(files):
        IMG_NAME_2 = f
        if f == '000000.png':
            IMG_NAME_1 = IMG_NAME_2
            continue
        img_1, img_2 = read_image(IMG_DIR, IMG_NAME_1, IMG_NAME_2)
        ###SIFT###
        sift = cv2.xfeatures2d.SIFT_create()
        kp1, des1 = sift.detectAndCompute(img_1,None)
        kp2, des2 = sift.detectAndCompute(img_2,None)
        SIFT_1 = cv2.drawKeypoints(img_1, kp1, None, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        SIFT_2 = cv2.drawKeypoints(img_2, kp2, None, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        cv2.imshow('image1',SIFT_1)
        cv2.imshow('image2',SIFT_2)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        ###SIFT###
        
        ###FIND MATCHES###
        FLANN_INDEX_KDTREE = 0
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params,search_params)
        matches = flann.knnMatch(des1,des2,k=2)
        good = []
        pts1 = []
        pts2 = []
        for i,(m,n) in enumerate(matches):
            if m.distance < 0.8*n.distance:
                good.append(m)
                pts2.append(kp2[m.trainIdx].pt)
                pts1.append(kp1[m.queryIdx].pt)
        ###FIND MATCHES###
        
        ###FIND ESSENTIALMATRIX###
        pts1_E = np.asarray(pts1)
        pts2_E = np.asarray(pts2)
        E, mask = cv2.findEssentialMat(pts1_E, pts2_E)
        pts1_E = pts1_E[mask.ravel()==1]
        pts2_E = pts2_E[mask.ravel()==1]
        points, r, t, mask = cv2.recoverPose(E, pts1_E, pts2_E)
        ###FIND ESSENTIALMATRIX###
        
        ###CALCULATE R AND T###
        T += -R@t
        R = r@R
        ###CALCULATE R AND T###
        IMG_NAME_1 = IMG_NAME_2